<a href="https://colab.research.google.com/github/ritwin82/Auto_ml_pipeline/blob/main/neural_hack3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install xgboost

In [ ]:
import pandas as pd
import numpy as np
import xgboost as xgb
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split


train_final = pd.read_csv('train_final.csv')
hidden_history = pd.read_csv('hidden_dist_history.csv')
test_x = pd.read_csv('test_X_q1_2026.csv')


full_history = pd.concat([train_final, hidden_history], axis=0, ignore_index=True)


targets = ['Persondays_of_Central_Liability_so_far', 'Total_No_of_Works_Takenup']

base_features = [col for col in test_x.columns if col not in ['date', 'month', 'district_id', 'state_id']]


all_districts = pd.concat([full_history['district_id'], test_x['district_id']]).unique()
all_states = pd.concat([full_history['state_id'], test_x['state_id']]).unique()

le_dist = LabelEncoder().fit(all_districts)
le_state = LabelEncoder().fit(all_states)

full_history['district_enc'] = le_dist.transform(full_history['district_id'])
full_history['state_enc'] = le_state.transform(full_history['state_id'])
test_x['district_enc'] = le_dist.transform(test_x['district_id'])
test_x['state_enc'] = le_state.transform(test_x['state_id'])

features = base_features + ['district_enc', 'state_enc']


submission = test_x[['district_id', 'date']].copy()

for target in targets:

    model = xgb.XGBRegressor(
    objective='reg:squarederror',
    n_estimators=1000,
    learning_rate=0.03,    # Lower LR for better generalization
    max_depth=5,           # Prevent overfitting
    subsample=0.8,
    colsample_bytree=0.8,
    early_stopping_rounds=50,
    n_jobs=-1
)

    train_data_filtered = full_history.dropna(subset=[target])


    X = train_data_filtered[features]
    y = train_data_filtered[target]
    X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

    model.fit(X_train, y_train, eval_set=[(X_val, y_val)], verbose=False)


    preds = model.predict(test_x[features])


    if target == 'Persondays_of_Central_Liability_so_far':
        preds = np.maximum(preds, 0)

    submission[target] = preds

# 6. Save the results
submission.to_csv('submission.csv', index=False)